<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="25%"><img src="media/decartes.jpg"
alt="DeCART Icon" width="128" height="171"><br>
</td>
<td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">DeCART Summer School<br>
for<br>
Biomedical Data Science</font></h1></td>
<td valign="center" align="center" width="25%"><img
src="media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


# Working with Textual Data
#### &copy; Brian E. Chapman, Ph.D.


### To get the latest version of this lesson, execute the cell below

In [9]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob

In [16]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [17]:
rad_data = \
pd.read_sql("""SELECT * 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,icustay_id,elemid,charttime,realtime,cgid,correction,cuid,category,title,text,exam_name,patient_info
0,56,28766.0,NaN,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,None,None
1,56,28766.0,NaN,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,None,None
2,56,28766.0,NaN,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,None,None
3,56,28766.0,NaN,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,None,None
4,56,28766.0,NaN,None,2644-01-19 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,None,None


In [11]:
rad_data.shape

(10000, 14)

## Section Splitting

<img src="http://medicine.utah.edu/dbmi/images/faculty/Chapman,Wendy_240x360.jpg"
alt="Wendy Chapman" width="128">

Clinical reports are often divided into sections. Physicians are taught to write their reports in the SOAP format: subjective information, objective information, assessment, and plan, and sections often reflect this process. Information each setion may be formatted differently. For example, subjective information is often narrative, telling a story of the patient's description of their past medical history and their current issues. The objective section may be much more telegraphic with shortened descriptions of measurements like blood pressure and heart rate, such as "120/80 88". You may need different NLP techniques to accurately extract information out of different sections. Sentence splitting, for example, may be different in subjective versus objective sections. Radiology reports are not in the SOAP format, but section identification can still be very important, because in these reports the radiologist describes what she sees on the radiographic image then sometimes summarizes her impression of what the observations may represent, such as "the infiltrate could be consistent with pneumonia." For some tasks, you may only be interested in the impression, which has more weight than the reasoning that occurs in the Findings section.

## Exercise

Write a function that returns the impression section of a report. The function will take two arguments:

1. The text to search for the impression section is the first positional argument.
1. A list of phrases to use to identify the impression section.

Not every report will have a section labeled "IMPRESSION". You can consider "INTERPRETATION" and "CONCLUSIONS" as synonyms for "IMPRESSION"

If you cannot identify the impresison section (or equivalent), return an empty string.

#### Hints

* Use a for loop  to iterate across
* Use the find method of a string to identify where in the string the impression section occurs
* Use slicing to grab the impression section


In [53]:

def find_impression(txt,search_terms=["IMPRESSION","INTERPRETATION","CONCLUSIONS"]):
                for term in search_terms:
                    ind = txt.find(term)
                    if ind != -1:
                     return txt[ind:]  
                return ""

find_impression(txt10)

NameError: name 'txt10' is not defined

In [66]:
import numpy as np
def do_math(func):
    return [func(i) for i in np.arrange(-np.pi,np.pi,0.5)]

In [67]:
type(np.cos)

numpy.ufunc

## Exercise 

Use ``apply()`` to create a new column in the ``rad_data`` named "IMPRESSIONS" with the string identified with ``find_impression()``.

In [68]:
do_math(np.cos)

AttributeError: module 'numpy' has no attribute 'arrange'

In [50]:
rad_data["IMPRESSIONS"] = \
   rad_data["text"].apply(find_impression)
rad_data.head()
    

TypeError: tuple indices must be integers or slices, not str

## How else might we need to break up a report?

* How would I break a report into sentences?
* How would I break a report into words?

## Group Exercise

### Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame
* Use TextBlob to get words